# 目的
OpenMath2-Llama3.1-8Bに対して、apply_chat_templateを使うテスト

ref: 
- https://github.com/unslothai/unsloth
- https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing#scrollTo=6bZsfBuZDeCL
- https://huggingface.co/blog/mlabonne/sft-llama3
- https://huggingface.co/docs/trl/sft_trainer#accelerate-fine-tuning-2x-using-unsloth

In [1]:
# pip install "unsloth[cu121-torch230] @ git+https://github.com/unslothai/unsloth.git"
# -> だめ

# pip uninstall torch
# pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# pip install "unsloth[cu121-torch240] @ git+https://github.com/unslothai/unsloth.git"
# -> OK

In [2]:
# path setting
EXP_NAME = "e018-causal"
# MODEL_NAME = "unsloth/gemma-2-9b-it-bnb-4bit"
# MODEL_NAME = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
MODEL_NAME = "nvidia/OpenMath2-Llama3.1-8B"
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"

DATA_PATH = "data"
DATASET_NAME = f"{EXP_NAME}-{MODEL_NAME.split('/')[-1].replace('.', '-')}"
OUTPUT_PATH = f"output/causal/{EXP_NAME}"
MODEL_OUTPUT_PATH = f"{OUTPUT_PATH}/trained_model"
ENV_PATH = "env_file"

# experiment parameter
DEBUG = False
TRAINING = True
UPLOAD_DATA_TO_S3 = True
UPLOAD_DATA_TO_KAGGLE = True
REMOVE_LOCAL_FILE = False
WANDB = True

# USE_FOLD = 0
# USE_DATA_RATE = 1.0
# VALID_DATA_SIZE = 3000

# model parameter
TRAINING_MAX_LENGTH = 384  # 512
SEED = 42
EPOCH = 5
LR = 2e-04
TRAIN_BS = 1  # 2  # 4  # 16
GRAD_ACC_STEP = 128 // TRAIN_BS  # 仮想的なバッチサイズはTRAIN_BS * GRAD_ACC_STEPとなる
EVAL_BS = 1  # 2  # 4  # 16
# NUM_LABELS = 3

FREEZE_LAYERS = 24  # 16

# rola parameter
LORA_R = 8  # 16
LORA_ALPHA = LORA_R * 2
LORA_BIAS = "none"

RESUME_FROM_CHECKPOINT = False  # 途中から再開する場合はTrueにする

In [3]:
!nvidia-smi

In [4]:
!python --version

In [5]:
def resolve_path(base_path: str) -> str:
    import os

    cwd = os.getcwd()
    print(cwd)
    if cwd == f"/notebooks":
        print("Jupyter Kernel By VSCode!")
        return f"/notebooks/{COMPETITION_NAME}/{base_path}"
    elif cwd == f"/notebooks/{COMPETITION_NAME}":
        print("nohup!")
        return base_path
    elif cwd == f"/notebooks/{COMPETITION_NAME}/{COMPETITION_NAME}/exp":
        print("Jupyter Lab!")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/reranker":
        print("VastAi! Reranker")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/retriever":
        print("VastAi! Retriever")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}/exp/causal":
        print("VastAi! Causal")
        return f"../../{base_path}"
    elif cwd == f"/root/{COMPETITION_NAME}":
        print("VastAi!")
        return base_path
    else:
        raise Exception("Unknown environment")


DATA_PATH = resolve_path(DATA_PATH)
print(DATA_PATH)
OUTPUT_PATH = resolve_path(OUTPUT_PATH)
print(OUTPUT_PATH)
MODEL_OUTPUT_PATH = resolve_path(MODEL_OUTPUT_PATH)
print(MODEL_OUTPUT_PATH)
ENV_PATH = resolve_path(ENV_PATH)
print(ENV_PATH)

In [6]:
def validate_dataset_name(dataset_name: str) -> None:
    if len(dataset_name) < 6 or len(dataset_name) > 50:
        raise Exception(
            f"データセットの文字列は6~50文字にしてください。現在{len(DATASET_NAME)}文字"
        )
    if "_" in dataset_name or "." in dataset_name:
        raise Exception("datasetの名称に'_'、'.'の使用は禁止です")


validate_dataset_name(DATASET_NAME)

# import

In [7]:
import os
import random

import polars as pl
import numpy as np

import wandb

import torch
from datasets import Dataset
from transformers import TrainingArguments

from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported

In [8]:
os.environ["TOKENIZERS_PARALLELISM"] = "true"
NUM_PROC = 16  # os.cpu_count()

In [9]:
import torch
import transformers
import datasets
import trl

assert torch.__version__ == "2.4.0+cu121"
assert transformers.__version__ == "4.44.2"
assert datasets.__version__ == "3.0.0"
assert trl.__version__ == "0.11.1"

In [10]:
# Seed the same seed to all
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True


seed_everything(SEED)

In [11]:
from dotenv import load_dotenv

load_dotenv(f"{ENV_PATH}/.env")

True

# Wandb

In [12]:
if WANDB:
    wandb.login(key=os.environ["WANDB_API_KEY"])
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

'wandb'

# Model Tokenizer Load

In [13]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_NAME,
    max_seq_length=TRAINING_MAX_LENGTH,
    dtype=None,
    load_in_4bit=True,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
# https://www.kaggle.com/code/kishanvavdara/lmsys-llama-3-tpu-train
# tokenizer.padding_side = "right"

# Data Import & Preprocess

In [15]:
train_long = pl.read_csv(f"{DATA_PATH}/train_long.csv").filter(
    (pl.col("CorrectAnswer") != pl.col("AnswerAlphabet"))
    & (pl.col("MisconceptionId").is_not_null())
)

In [16]:
if DEBUG:
    train_long = train_long.head(100)

In [17]:
train_long

QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,CorrectAnswerText,AnswerType,AnswerText,AllText,AnswerAlphabet,QuestionId_Answer,MisconceptionId,MisconceptionName
i64,str,str,str,str,str,str,str,str,str,str,i64,str
0,"""Use the order of operations to…","""BIDMAS""","""\[ 3 \times 2+4-5 \] Where do …","""A""","""\( 3 \times(2+4)-5 \)""","""AnswerDText""","""Does not need brackets""","""Use the order of operations to…","""D""","""0_D""",1672,"""Confuses the order of operatio…"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerAText""","""\( t \)""","""Simplify an algebraic fraction…","""A""","""1000_A""",891,"""Incorrectly cancels what they …"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerCText""","""\( 1 \)""","""Simplify an algebraic fraction…","""C""","""1000_C""",891,"""Incorrectly cancels what they …"
1000,"""Simplify an algebraic fraction…","""Simplifying Algebraic Fraction…","""Simplify the following, if pos…","""B""","""\( -1 \)""","""AnswerDText""","""Does not simplify""","""Simplify an algebraic fraction…","""D""","""1000_D""",353,"""Does not recognise when one pa…"
1001,"""Round numbers to two decimal p…","""Rounding to Decimal Places""","""What is \( \mathbf{3 . 5 1 6 3…","""B""","""\( 3.52 \)""","""AnswerAText""","""\( 3.51 \)""","""Round numbers to two decimal p…","""A""","""1001_A""",1379,"""Rounds down instead of up"""
…,…,…,…,…,…,…,…,…,…,…,…,…
99,"""Given the perimeter, work out …","""Volume and Capacity Units""","""A regular pentagon has a total…","""C""","""\( 3.4 \mathrm{~cm} \)""","""AnswerBText""","""\( 12 \mathrm{~cm} \)""","""Given the perimeter, work out …","""B""","""99_B""",1815,"""When given the perimeter of a …"
99,"""Given the perimeter, work out …","""Volume and Capacity Units""","""A regular pentagon has a total…","""C""","""\( 3.4 \mathrm{~cm} \)""","""AnswerDText""","""Not enough information""","""Given the perimeter, work out …","""D""","""99_D""",255,"""Does not know the meaning of t…"
9,"""Identify horizontal translatio…","""Transformations of functions i…","""What transformation maps the g…","""C""","""Translation by vector \( \left…","""AnswerAText""","""Translation by vector \( \left…","""Identify horizontal translatio…","""A""","""9_A""",1889,"""Believes f(x - a) translates t…"


In [18]:
# messages = [
#     {
#         "role": "system",
#         "content": "You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.",
#     },
#     {"role": "user", "content": "Who are you?"},
# ]

# # TODO: CoT(think step by step)をどうFine-Tuningに反映させるべきかを調べる
# alpaca_prompt = """### Input:
# Question: {question_text}
# Incorrect Answer: {incorrect_answer_text}
# Correct Answer: {correct_answer_text}
# Construct Name: {construct_name}
# Subject Name: {subject_name}

# ### Response:
# {misconception_name}"""

# EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


# def formatting_prompts_func(examples):
#     texts = []
#     for (
#         question_text,
#         incorrect_answer_text,
#         correct_answer_text,
#         construct_name,
#         subject_name,
#         misconception_name,
#     ) in zip(
#         examples["QuestionText"],
#         examples["AnswerText"],  # incorrect_answer_text
#         examples["CorrectAnswerText"],
#         examples["ConstructName"],
#         examples["SubjectName"],
#         examples["MisconceptionName"],
#     ):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = (
#             alpaca_prompt.format(
#                 question_text=question_text,
#                 incorrect_answer_text=incorrect_answer_text,
#                 correct_answer_text=correct_answer_text,
#                 construct_name=construct_name,
#                 subject_name=subject_name,
#                 misconception_name=misconception_name,
#             )
#             + EOS_TOKEN
#         )
#         texts.append(text)
#     return {
#         "text": texts,
#     }


# train = Dataset.from_polars(train_long).map(
#     formatting_prompts_func,
#     batched=True,
# )

In [19]:
QUESTION_TEMPLATE = """Question: {question_text}
Incorrect Answer: {incorrect_answer_text}
Correct Answer: {correct_answer_text}
Construct Name: {construct_name}
Subject Name: {subject_name}"""


class ApplyTemplateForTrain:
    def make_masaages(self, question: str, answer: str) -> list[dict[str, str]]:
        return [
            {
                "role": "system",
                "content": "You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.",
            },
            {"role": "user", "content": question},
            {"role": "assistant", "content": answer},
        ]

    def apply_template(self, examples):
        messages = []
        for (
            question_text,
            incorrect_answer_text,
            correct_answer_text,
            construct_name,
            subject_name,
            misconception_name,
        ) in zip(
            examples["QuestionText"],
            examples["AnswerText"],  # incorrect_answer_text
            examples["CorrectAnswerText"],
            examples["ConstructName"],
            examples["SubjectName"],
            examples["MisconceptionName"],
        ):
            message = self.make_masaages(
                question=QUESTION_TEMPLATE.format(
                    question_text=question_text,
                    incorrect_answer_text=incorrect_answer_text,
                    correct_answer_text=correct_answer_text,
                    construct_name=construct_name,
                    subject_name=subject_name,
                ),
                answer=misconception_name,
            )
            messages.append(message)

        text = [
            tokenizer.apply_chat_template(
                message, tokenize=False, add_generation_prompt=False
            )
            for message in messages
        ]

        return {"text": text}


train = Dataset.from_polars(train_long).map(
    ApplyTemplateForTrain().apply_template, batched=True
)

Map:   0%|          | 0/4370 [00:00<?, ? examples/s]

In [20]:
print(train[0]["text"])

In [21]:
print(train[10]["text"])

# Split Train Valid

In [22]:
train, valid = (
    train.filter(lambda x: x["QuestionId"] % 3 != 0, num_proc=NUM_PROC),
    train.filter(lambda x: x["QuestionId"] % 3 == 0, num_proc=NUM_PROC),
)

Filter (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

Filter (num_proc=16):   0%|          | 0/4370 [00:00<?, ? examples/s]

# Model Predict Check Before Training

In [23]:
print(" === Model Predict Check Before Training Start ===")

In [24]:
# EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN


# def formatting_prompts_func_predict(examples):
#     texts = []
#     for (
#         question_text,
#         incorrect_answer_text,
#         correct_answer_text,
#         construct_name,
#         subject_name,
#     ) in zip(
#         examples["QuestionText"],
#         examples["AnswerText"],  # incorrect_answer_text
#         examples["CorrectAnswerText"],
#         examples["ConstructName"],
#         examples["SubjectName"],
#     ):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = (
#             alpaca_prompt.format(
#                 question_text=question_text,
#                 incorrect_answer_text=incorrect_answer_text,
#                 correct_answer_text=correct_answer_text,
#                 construct_name=construct_name,
#                 subject_name=subject_name,
#                 misconception_name="",
#             )
#             + EOS_TOKEN
#         )
#         texts.append(text)
#     return {
#         "text": texts,
#     }


# valid_for_predict = valid.map(
#     formatting_prompts_func_predict,
#     batched=True,
# )

In [25]:
class ApplyTemplateForPredict:
    def make_masaages(self, question: str) -> list[dict[str, str]]:
        return [
            {
                "role": "system",
                "content": "You are a mathematics expert. You are given a mathematics question, an incorrect answer, a correct answer, a construct name, and a subject name. Using these, identify what misconception exists in the incorrect answer.",
            },
            {"role": "user", "content": question},
        ]

    def apply_template(self, examples):
        messages = []
        for (
            question_text,
            incorrect_answer_text,
            correct_answer_text,
            construct_name,
            subject_name,
        ) in zip(
            examples["QuestionText"],
            examples["AnswerText"],  # incorrect_answer_text
            examples["CorrectAnswerText"],
            examples["ConstructName"],
            examples["SubjectName"],
        ):
            message = self.make_masaages(
                question=QUESTION_TEMPLATE.format(
                    question_text=question_text,
                    incorrect_answer_text=incorrect_answer_text,
                    correct_answer_text=correct_answer_text,
                    construct_name=construct_name,
                    subject_name=subject_name,
                )
            )
            messages.append(message)

        text = [
            tokenizer.apply_chat_template(
                message, tokenize=False, add_generation_prompt=True
            )
            for message in messages
        ]

        return {"text": text}


valid_for_predict = valid.map(ApplyTemplateForPredict().apply_template, batched=True)

Map:   0%|          | 0/1460 [00:00<?, ? examples/s]

In [26]:
print(valid_for_predict[0]["text"])

In [27]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference


def predict(valid: Dataset, idx: int, model):
    inputs = tokenizer(
        valid["text"][idx],
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=64, do_sample=True)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
    print("\n### Answer Misconception Name:")
    print(valid["MisconceptionName"][idx])

In [28]:
print(valid_for_predict[0]["text"])

In [29]:
predict(valid_for_predict, 0, model)

In [30]:
predict(valid_for_predict, 1, model)

In [31]:
predict(valid_for_predict, 2, model)

In [32]:
print(" === Model Predict Check Before Training End ===")

# Model

In [33]:
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,  # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        # "o_proj",
        # "gate_proj",
        # "up_proj",
        # "down_proj",
        "embed_tokens",
        "lm_head",
    ],
    lora_alpha=LORA_ALPHA,
    lora_dropout=0,  # Supports any, but = 0 is optimized
    bias="none",  # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing="unsloth",  # True or "unsloth" for very long context
    random_state=SEED,
    use_rslora=False,  # We support rank stabilized LoRA
    loftq_config=None,  # And LoftQ
    layers_to_transform=[i for i in range(32) if i >= FREEZE_LAYERS],
)

In [34]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): ModulesToSaveWrapper(
          (original_module): Embedding(128256, 4096)
          (modules_to_save): ModuleDict(
            (default): Embedding(128256, 4096)
          )
        )
        (layers): ModuleList(
          (0-23): 24 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
              (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
              (rotary_emb): LlamaExtendedRotaryEmbedding()
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (up_proj): Linear4bit(in_featur

In [35]:
training_arguments = TrainingArguments(
    output_dir=MODEL_OUTPUT_PATH,
    learning_rate=LR,
    per_device_train_batch_size=TRAIN_BS,
    gradient_accumulation_steps=GRAD_ACC_STEP,
    eval_accumulation_steps=GRAD_ACC_STEP,
    per_device_eval_batch_size=EVAL_BS,
    num_train_epochs=EPOCH,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=0.1,
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=1,
    logging_steps=2,
    seed=SEED,
    metric_for_best_model="eval_loss",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine_with_restarts",
    report_to=REPORT_TO,
    run_name=EXP_NAME,
    load_best_model_at_end=True,
    fp16=True,
    fp16_full_eval=True,
    gradient_checkpointing=True,
)

In [36]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train,
    eval_dataset=valid,
    dataset_text_field="text",
    max_seq_length=TRAINING_MAX_LENGTH,
    dataset_num_proc=NUM_PROC,
    packing=False,  # Can make training 5x faster for short sequences.
    args=training_arguments,
)

Map (num_proc=16):   0%|          | 0/2910 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/1460 [00:00<?, ? examples/s]

In [37]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [38]:
MODEL_OUTPUT_PATH

'output/causal/e018-causal/trained_model'

In [39]:
if TRAINING:
    trainer_stats = trainer.train()
else:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=MODEL_OUTPUT_PATH + "/lora_model",
        max_seq_length=TRAINING_MAX_LENGTH,
        dtype=None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
        # load_in_4bit=True,  # Use 4bit quantization to reduce memory usage. Can be False.
    )

Counting untrained tokens:   0%|          | 0/2910 [00:00<?, ? examples/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1002.00 MiB. GPU 0 has a total capacity of 15.77 GiB of which 599.12 MiB is free. Process 1356248 has 15.18 GiB memory in use. Of the allocated memory 14.69 GiB is allocated by PyTorch, and 101.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

# Predict Valid

In [ ]:
print(" === Model Predict Check After Training Start ===")

In [42]:
predict(valid_for_predict, 0, model)

In [43]:
predict(valid_for_predict, 1, model)

In [44]:
predict(valid_for_predict, 2, model)

In [ ]:
print(" === Model Predict Check After Training End ===")

# Save

In [45]:
model.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")  # Local saving
tokenizer.save_pretrained(f"{MODEL_OUTPUT_PATH}/lora_model")

('output/causal/e014-causal/trained_model/lora_model/tokenizer_config.json',
 'output/causal/e014-causal/trained_model/lora_model/special_tokens_map.json',
 'output/causal/e014-causal/trained_model/lora_model/tokenizer.json')

# AWSへのアップロード

In [46]:
# S3へのアップロード
if not DEBUG and UPLOAD_DATA_TO_S3:
    # uninstall
    !sudo rm /usr/bin/aws
    !sudo rm /usr/bin/aws_completer
    !sudo rm -rf /usr/local/aws-cli

    # install
    !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
    !unzip -o -qq awscliv2.zip
    !sudo ./aws/install --update

    # upload
    output_name = MODEL_OUTPUT_PATH.split("/")[-1]
    os.system(
        f"aws s3 cp --recursive {MODEL_OUTPUT_PATH} s3://{COMPETITION_NAME}/trained_model/{output_name}"
    )

In [47]:
# ダウンロード（参考）
# !sudo rm /usr/bin/aws
# !sudo rm /usr/bin/aws_completer
# !sudo rm -rf /usr/local/aws-cli

# !curl "https://awscli.amazonaws.com/awscli-exe-linux-x86_64.zip" -o "awscliv2.zip"
# !unzip -o -qq awscliv2.zip
# !sudo ./aws/install --update

# !aws s3 cp --recursive s3://automated-essay-scoring/trained_model/e005-regression /notebooks/automated_essay_scoring/trained_models/e005-regression

# Kaggle Datasetへのupload

In [12]:
import os

os.system("mkdir -p ~/.kaggle/")
os.system(f"cp {ENV_PATH}/kaggle.json ~/.kaggle/")
os.system("chmod 600 ~/.kaggle/kaggle.json")

0

In [13]:
if not DEBUG and UPLOAD_DATA_TO_KAGGLE:
    import os
    import json

    from kaggle.api.kaggle_api_extended import KaggleApi

    def dataset_create_new(dataset_name: str, upload_dir: str):
        dataset_metadata = {}
        dataset_metadata["id"] = f"sinchir0/{dataset_name}"
        dataset_metadata["licenses"] = [{"name": "CC0-1.0"}]
        dataset_metadata["title"] = dataset_name
        with open(os.path.join(upload_dir, "dataset-metadata.json"), "w") as f:
            json.dump(dataset_metadata, f, indent=4)
        api = KaggleApi()
        api.authenticate()
        api.dataset_create_new(folder=upload_dir, convert_to_csv=False, dir_mode="tar")

    print(f"Create Dataset name:{DATASET_NAME}, output_dir:{MODEL_OUTPUT_PATH}")
    dataset_create_new(dataset_name=DATASET_NAME, upload_dir=MODEL_OUTPUT_PATH)

Create Dataset name:e017-causal-Meta-Llama-3-1-8B-Instruct-bnb-4bit, output_dir:../../output/causal/e017-causal/trained_model
Starting upload for file checkpoint-110.tar


100%|██████████| 62.9M/62.9M [00:03<00:00, 20.2MB/s]


Upload successful: checkpoint-110.tar (63MB)
Starting upload for file lora_model.tar


100%|██████████| 26.8M/26.8M [00:02<00:00, 13.9MB/s]


Upload successful: lora_model.tar (27MB)


# ローカルからのデータの削除

In [50]:
# if not DEBUG and REMOVE_LOCAL_FILE:
#     # ローカルからは削除
#     os.system(f"rm -rf {MODEL_OUTPUT_PATH}")

In [51]:
if WANDB:
    wandb.finish()

In [52]:
print("finish Notebook!")